In [26]:
import sys
sys.path.append("../")

import pandas as pd

In [27]:
from config_utils.load_config import load_params_from_yaml, DataParamsSchema
dataset_params = load_params_from_yaml("dataset_params.yaml", DataParamsSchema)

from pathlib import Path
root_dir = Path().resolve().parents[0]

In [3]:
train_df = pd.read_csv(root_dir / dataset_params.data_params.train_data_path)
test_df = pd.read_csv(root_dir / dataset_params.data_params.test_data_path)
subred_1 = pd.read_csv(root_dir / dataset_params.data_params.subset1_path /  (dataset_params.data_params.subreddit1 + '.csv'))
subred_2 = pd.read_csv(root_dir / dataset_params.data_params.subset2_path /  (dataset_params.data_params.subreddit2 + '.csv'))


In [4]:
train_df.head()

,title,body,subreddit
0,Looking for a good pair of brown shoes.,Looking for a good pair of causal brown shoes ...,malefashionadvice
1,DAE collect vintage boomboxes?,"I just picked up an awesome new box, and decid...",DoesAnybodyElse
2,Australia to Europe: Where to stop for a month...,Hey! So I'm planning to go to Europe indefinit...,travel
3,Hey /r/fit! I've not posted here before so bea...,"A bit of context, I'm thin. Too thin I would s...",Fitness
4,"Why does Firefox keep doing this, and how can ...",I guess I'm not a super computer wiz or anythi...,AskReddit


In [5]:
test_df.head()


,title,body,subreddit
0,How do you get inappropriate feelings for some...,Hi guys\n\nI write to you today quite torn up ...,relationship_advice
1,"Looking for more bands to listen to, my absolu...",So Deathcab has been my favorite for at least ...,Music
2,Going to get an abortion and need advice on me...,"I just need some questions answered, so I know...",TwoXChromosomes
3,How can I connect with my Grandpa? He's in his...,"I'm somewhat of a new Redditor, but I've reall...",AskReddit
4,Hey Reddit want to help me with a self-fulfill...,I bet if I write something obnoxious about the...,reddit.com


In [6]:
subred_1.head()

,title,body,subreddit
0,"Programmers, do you put your comments (before|...",Inspired by: \n\nIn the several years I've bee...,programming
1,Anyone want to give some good 'finding a job' ...,I'm approaching the end of my junior year of c...,programming
2,I cranked out an awesome game in 48 hours for ...,The global game jam went down this past weeken...,programming
3,The best way to store passwords in your applic...,...is only as good as the policy you enforce. ...,programming
4,"Hey Proggit here is my first program, a mixtur...",It's an application that emails you when your ...,programming


In [7]:
subred_2.head()

,title,body,subreddit
0,Here's an idea: Left 4 Dead Jurassic Park,After the power to the park goes out and the d...,gaming
1,"r/gaming, I'm looking for a good quality heads...",I've been using this crappy little headset tha...,gaming
2,Please help me figure out why it's so tough fo...,"4 times out of 5, when I try to connect to a V...",gaming
3,How to rob your opponent of a kill in MW2 in l...,Its quite easy. All you have to do is drop a g...,gaming
4,Anybody else find it hard to open a game after...,"I don't know if I'm weird or what, but every t...",gaming


In [8]:
len(train_df), len(test_df), len(subred_1), len(subred_2)

(60000, 6000, 1000, 1000)

In [9]:
# Проверяем на дубликаты между всеми парами датафреймов
def check_duplicates(df1, df2, name1, name2):
    col = "body"
    duplicates = df1.merge(df2, on=col, how='inner')

    # Если дубликаты найдены, выводим их
    if not duplicates.empty:
        print("Найдены дубликаты по столбцу 'B':")
        print(duplicates)

        # Находим индексы дубликатов в df1
        df1_duplicates_indices = df1.index[df1[col].isin(duplicates[col])].tolist()
        
        # Находим индексы дубликатов в df2
        df2_duplicates_indices = df2.index[df2[col].isin(duplicates[col])].tolist()

        print(f"Индексы дубликатов в df1: {df1_duplicates_indices}")
        print(f"Индексы дубликатов в df2: {df2_duplicates_indices}")
    else:
        print("Дубликатов нет.")

check_duplicates(train_df, test_df, "train_df", "test_df")
check_duplicates(train_df, subred_1, "train_df", "subred_1")
check_duplicates(train_df, subred_2, "train_df", "subred_2")
check_duplicates(test_df, subred_1, "test_df", "subred_1")
check_duplicates(test_df, subred_2, "test_df", "subred_2")
check_duplicates(subred_1, subred_2, "subred_1", "subred_2")


Дубликатов нет.
Дубликатов нет.
Дубликатов нет.
Дубликатов нет.
Дубликатов нет.
Дубликатов нет.


In [10]:
col = "body"
duplicates = train_df.merge(test_df, on=col, how='inner')
duplicates

,title_x,body,subreddit_x,title_y,subreddit_y


In [17]:
type(duplicates)

pandas.core.frame.DataFrame

In [13]:
duplicates = test_df[test_df.duplicated(keep=False)]

# Masked

In [28]:
import torch
inputs_loaded = torch.load(root_dir / 'data/masked/gaming.pt')

/tmp/ipykernel_107890/1108389198.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  inputs_loaded = torch.load(root_dir / 'data/masked/gaming.pt')


In [29]:
inputs_loaded["input_ids"][5]

tensor([  101,  2061,  2651,   103,  2001,   103,  2083,   103,  3466,   103,
         1010,  2383,  1037,  2204,  2214,  2051,  1010,  2043,  3402,  1996,
         2208,  8007,  1012,  2582,  4812,  3662,  2008,   103,  8007,  2588,
          103,  2000,  7170,  1037,  2047,  2930,  1997,  1996,  2208,  1012,
         1045,  2253,  1998,  2246,  3784,  2005,  1996,  4949,  5371,  2000,
         5672,  2026, 27279,  2028,  1010,  2021,  1045,  2481,  1005,  1056,
         2424,   103,  1998,  5295,  2870,  2000,  2417,   103, 18570,  1998,
        19222,  9080,  2989,   103,   103,  1012,  2174,  1010,  1045,  3603,
         2019,  5724,   103,   103,  5144,   103,   103, 20410, 11109,  1997,
         2334, 17053,  1012,  1000,  2588,  2770,  2008,  5724,  1010,  5492,
         7039,  2041,  2026,  5361,  2208,  6764,  1010,  2988,  2008,  2009,
          103,  2179,  2416, 13593,  6764,  1010,  1998,  8073, 22817,  2216,
         2416,  6764,  1012,  2009,  2165,  2672,  1019,   102])